In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from scipy import stats
sns.set()

In [48]:
"""
Leitura dos dataframes de base e target.
Define as métricas que devem ser analisadas.
"""

df_base = pd.read_csv('/content/data_base.csv')
df_target = pd.read_csv('/content/data_target.csv')

possible_metrics = ['cpu_app', 'memory_app', 'cpu_db', 'memory_db', 'rows_fetched', 'rows_returned', 'db_commits', 'response_time']

In [49]:
"""
Remove colunas não utilizadas não utilizadas do csv.
"""
removable_columns = ['timestamp_begin', 'timestamp_end', 'total_calls']
df_base.drop(removable_columns, axis=1, inplace=True)
df_target.drop(removable_columns, axis=1, inplace=True)

In [50]:
"""
A função para recuperar os outliers está utilizando a regra iqr (inter quartil)
No nosso caso, encontramos que a remoção dos outliers abaixo do minimo definido
referentes as chamadas totais faz mais sentido, justamente por remover os pontos
de dados da amostragem que não nos interessa, que seria o ínicio e fim do teste,
onde nem os caches, nem os buffers foram definidos ainda. Além disso, os dados acima
do máximo definido pode representar sobrecarga no teste, gerando um estresse, que pode
impactar negativamente nas amostras coletadas.
"""
def recupera_outliers(df, target):
  outliers_indexes = []

  q1 = df[target].quantile(0.25)
  q3 = df[target].quantile(0.75)
  iqr = q3-q1
  maximum = q3 + (1.5 * iqr)
  minimum = q1 - (1.5 * iqr)
  outlier_samples = df[(df[target] < minimum) | (df[target] > maximum)]
  outliers_indexes.extend(outlier_samples.index.tolist())

  outliers_indexes = list(set(outliers_indexes))
  return outliers_indexes

In [51]:
"""
Recupera regressão linear utilizando OLS para exibir valores do R².
"""

def recupera_regressao_linear(df, target='cpu'):
  x = sm.add_constant(df.drop(possible_metrics, axis=1))

  results = sm.OLS(df[target], x).fit()

  return results.summary()

In [ ]:
"""
Recupera o R² ajustado para a regressão linear do conjunto de dados olhando
para as possíveis métricas. Apenas no conjunto base que é o responsável pela escala
Seleciona métricas que possuem R² ajustado maior ou igual a 0.67.
"""
analyzable_metrics = []
for metric in possible_metrics:
  adjustedR2 = recupera_regressao_linear(df_base, metric).tables[0].data[1][3].strip()
  print("Métrica: ", metric)
  print("R² Ajustado: ", adjustedR2)
  if float(adjustedR2) >= 0.67:
    analyzable_metrics.append(metric)

print('\nMétricas passíveis de análise:')
print(analyzable_metrics)

In [53]:
"""
Faz o plot do gráfico de comparação entre dois conjuntos de dados,
olhando o parâmetro de métrica enviado.
"""

# Plote do gráfico de acordo com a métrica selecionada

def plot_grafico(df_base, df_target, metric):
  plt.plot(df_base[metric], 'b-', label='Base')
  plt.plot(df_target[metric], 'r--', label='Target')
  plt.legend(loc='lower right')
  plt.xlabel('Período de tempo (minuto)', fontsize = 14)
  plt.ylabel(f'{metric}', fontsize = 14)

  plt.show()

In [54]:
"""
Recupera os coeficientes das variáveis independentes (chamadas aos endpoints) e
dependente (valor da métrica) utilizando uma regressão linear múltipla com o
algoritmo de OLS (Ordinary Least Squares) [https://en.wikipedia.org/wiki/Ordinary_least_squares]
Obtivemos melhores resultados a partir da utilização desse algoritmo em comparação com o
sklearn LinearRegressionModel.
"""

def recupera_coeficientes(df, target):
  x = sm.add_constant(df.drop(possible_metrics, axis=1))

  results = sm.OLS(df[target], x).fit()

  # Coef da variável dependente
  intercept = float(results.summary().tables[1].data[1][1].strip())

  coef_independente = []
  for i in range(23):
    coef_independente.append(float(results.summary().tables[1].data[i+2][1].strip()))

  return intercept, coef_independente

In [55]:
"""
Recupera a soma dos coeficientes para ser utilizado na equação que aplica a escala
dos dados a partir da relação entre impacto da carga nos valores das métricas.
"""

def recupera_soma_coeficientes(df, intercept, coef_independente, index_line):

  return sum([value*df.iloc[index_line][index] for index, value in enumerate(coef_independente)])+intercept

In [ ]:
"""
Traz para escala do base todas as métricas de desempenho do conjunto de dados target dos testes.
Sobreescreve os valores das métricas em um novo dataframe de dados que estão em escala.
"""

df_target_scaled = df_target.copy()

for metric in analyzable_metrics:
  intercept_base, coef_independente_base = recupera_coeficientes(df_base, metric)

  for index, Mb in enumerate(df_base[metric]):
    df_target_scaled[metric][index] = (
      Mb * (
        recupera_soma_coeficientes(df_target, intercept_base, coef_independente_base, index)/recupera_soma_coeficientes(df_base, intercept_base, coef_independente_base, index)
      )
    )

In [57]:
df_base.name = 'Teste base'
df_target.name = 'Teste alvo'
df_target_scaled.name = 'Teste base normalizado'

In [58]:
def plot_grafico(df_base, df_target, metric, ax):
  ax.plot(df_base[metric], 'b-', label=df_base.name)
  ax.plot(df_target[metric], 'r--', label=df_target.name)
  ax.legend(loc='lower right')
  ax.set_xlabel('Período de tempo (minuto)', fontsize = 14)
  ax.set_ylabel(f'{metric}', fontsize = 14)

In [ ]:
"""
Realiza a impressão dos gráficos de comparação entre as métricas em escala e normais.
"""
fig, ax = plt.subplots(len(analyzable_metrics), 3, figsize=(25,25))
for index, metric in enumerate(analyzable_metrics):
  plot_grafico(df_base, df_target, metric, ax[index, 0])
  plot_grafico(df_base, df_target_scaled, metric, ax[index, 1])
  plot_grafico(df_target_scaled, df_target, metric, ax[index, 2])
plt.tight_layout()
plt.show()

In [60]:
"""
Remove os outliers de ambos conjuntos de dados (base e target) para cada métrica individualmente.
Para manter o tamanho da amostra, são removidos os índices de outliers tanto encontrados na
amostra do teste base, quanto target. O corte IQR feito garante um conjunto de dados mais
normal do que o original, satisfazendo o pressuposto do gráfico de controle de normalidade
nos outputs.
"""
dfs_target_scaled = {}
dfs_target = {}
for metric in analyzable_metrics:
  dfs_target[metric] = df_target.copy()
  dfs_target_scaled[metric] = df_target_scaled.copy()
  outliers_indexes = list(set([*recupera_outliers(dfs_target[metric], metric), *recupera_outliers(dfs_target_scaled[metric], metric)]))
  dfs_target[metric].drop(outliers_indexes, inplace=True)
  dfs_target[metric].reset_index(drop=True, inplace=True)

  dfs_target_scaled[metric].drop(outliers_indexes, inplace=True)
  dfs_target_scaled[metric].reset_index(drop=True, inplace=True)

In [ ]:
"""
Faz o plote dos gráficos de controle cuja distribuição base e alvo estão normais.
Verificação é feita usando o teste de Shapiro-Wilk. Limites inferiores e superiores
definidos a partir dos percentis 5 e 95 do conjunto de dados base.
"""
LSC = {} # Limite superior de controle
LIC = {} # Limite inferior de controle

for metric in analyzable_metrics:
  # Verifica normalidade a partir do shapiro (Se pvalue < 0.05, conjunto de dados não normal)
  _, pvalue_base = stats.shapiro(dfs_target[metric][metric])
  _, pvalue_target_scaled = stats.shapiro(dfs_target_scaled[metric][metric])
  if pvalue_base < 0.05 or pvalue_target_scaled < 0.05:
    continue

  df_base_statistic = dfs_target_scaled[metric].describe(percentiles=[.05, .95])

  # Plot do gráfico de controle
  fig, ax = plt.subplots(figsize = (10, 6))

  # Criação do eixo principal
  ax.plot(dfs_target[metric][metric], linestyle='-', marker='o', color='blue', label='Alvo')

  # Criação do limite superior
  LSC[metric] = df_base_statistic[metric]['95%']
  ax.axhline(LSC[metric], color='red')

  # Criação do limite inferior
  LIC[metric] = df_base_statistic[metric]['5%']
  ax.axhline(LIC[metric], color='red')

  # Criação da linha de média
  ax.axhline(df_base_statistic[metric]['mean'], color='green')

  # Título do gráfico
  ax.set_title(f'Gráfico de controle ({metric})')

  # Nomeação dos eixos
  ax.set(xlabel='Período de tempo (minuto)', ylabel=metric)

  # Nomeação lateral do gráfico a partir do limite do eixo x
  left, right = ax.get_xlim()
  ax.text(right + 0.3, LSC[metric], "LSC = " + str("{:.2f}".format(LSC[metric])), color='red')
  ax.text(right + 0.3, df_base_statistic[metric]['mean'], r'$\bar{x}$' + " = " + str("{:.2f}".format(df_base_statistic[metric]['mean'])), color='green')
  ax.text(right + 0.3, LIC[metric], "LIC = " + str("{:.2f}".format(LIC[metric])), color='red')

In [ ]:
"""
Verifica se os gráficos estão dentro ou fora de controle a partir do threshold estabelecido
de 20% para a taxa de violação do gráfico. Esse threshold é escolhido devido aos cortes
feitos no conjunto de dados para definição dos limites superiores e inferiores. No caso,
os limites inferiores e superiores são definidos a partir do 5º e do 95º percentil do dataframe,
portanto, o threshold tem que ser acima de 10% pelo menos, entre 15 a 20% é o ideal.
Como estamos observando diversos endpoints, encontrar uma alteração significativa de desempenho
pode ser um pouco díficil, porém, tem a questão do possível erro gerado a partir da escala das informações,
portanto, estamos utilizando 20% podendo caber uma avaliação quanto
a pontos de estatística descritiva dos dataframes, afim de invalidar dados imprecisos.
Dados imprecisos são dados providos de escalas feitas sob uma relação que não representa linearidade,
normalmente, são relações que possuem um baixo R².
"""

for index, metric in enumerate(analyzable_metrics):

  # Verifica normalidade a partir do shapiro (Se pvalue < 0.05, conjunto de dados não normal)
  _, pvalue_base = stats.shapiro(dfs_target[metric][metric])
  _, pvalue_target_scaled = stats.shapiro(dfs_target_scaled[metric][metric])
  if pvalue_base < 0.05 or pvalue_target_scaled < 0.05:
    continue

  sum_of_violation = 0
  for metric_data in dfs_target[metric][metric]:
    if metric_data > LSC[metric] or metric_data < LIC[metric]:
      sum_of_violation += 1

  print(f"Métrica: {metric}")
  print(f"Número de violações: {sum_of_violation}")
  print(f"Número total de amostras: {dfs_target_scaled[metric][metric].count()}")

  violation_ratio = sum_of_violation / dfs_target_scaled[metric][metric].count()
  threshold = 0.20 #

  print(f"Taxa de violação: {violation_ratio}\nThreshold pré-definido: {threshold}")
  if violation_ratio > threshold:
    print('Gráfico fora de controle')
  else:
    print('Gráfico dentro do controle')

  # Pula linha entre resultados de métricas
  if index < len(possible_metrics) - 1:
    print('\n')

In [ ]:
"""
Resultado do teste de Shapiro Wilk para conjuntos de dados base e alvo já em escala e filtrado.
"""
for metric in analyzable_metrics:
  print(metric)
  print(f"ANTES ----- Base: {stats.shapiro(df_target_scaled[metric])} ------- Target: {stats.shapiro(df_target[metric])}")
  print(f"DEPOIS -----Base: {stats.shapiro(dfs_target_scaled[metric][metric])} ------- Target: {stats.shapiro(dfs_target[metric][metric])}")
  print()

In [ ]:
"""
Plote dos histogramas com a curva feita a partir estimativa de densidade por Kernel (KDE)
Conjunto de dados base escalado.
"""
for metric in analyzable_metrics:
  fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(14, 8))
  fig.suptitle("Antes                                x                                Depois")
  sns.histplot(df_target_scaled[metric], kde=True, ax=ax1)
  sns.histplot(dfs_target_scaled[metric][metric], kde=True, ax=ax2)
  q1 = np.percentile(df_target_scaled[metric], 25)
  q3 = np.percentile(df_target_scaled[metric], 75)
  iqr = q3 - q1
  ax1.axvline(q1 - (1.5*iqr), color='red', ls='--', label='LIC', ymin = 0, ymax = 0.95 )
  ax1.axvline(q3 + (1.5*iqr), color='green', ls='--', label='LSC', ymin = 0, ymax = 0.95 )
  plt.show()

In [ ]:
"""
Plote dos histogramas com a curva feita a partir estimativa de densidade por Kernel (KDE)
Conjunto de dados alvo.
"""
for metric in analyzable_metrics:
  fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(14, 8))
  fig.suptitle("Antes                                x                                Depois")
  sns.histplot(df_target[metric], kde=True, ax=ax1)
  sns.histplot(dfs_target[metric][metric], kde=True, ax=ax2)
  q1 = np.percentile(df_target[metric], 25)
  q3 = np.percentile(df_target[metric], 75)
  iqr = q3 - q1
  ax1.axvline(q1 - (1.5*iqr), color='red', ls='--', label='LIC', ymin = 0, ymax = 0.95 )
  ax1.axvline(q3 + (1.5*iqr), color='green', ls='--', label='LSC', ymin = 0, ymax = 0.95 )
  plt.show()

In [ ]:
"""
Gráficos qq do conjunto de dados base escalado [target_scaled] (antes x depois).
"""
for metric in analyzable_metrics:
  fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(14, 8))
  fig.suptitle("Antes                                x                                Depois")
  sm.qqplot(df_target_scaled[metric], ax=ax1)
  sm.qqplot(dfs_target_scaled[metric][metric], ax=ax2)
  plt.show()

In [ ]:
"""
Gráficos qq do conjunto de dados alvo (antes x depois).
"""
for metric in analyzable_metrics:
  fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(14, 8))
  fig.suptitle("Antes                                x                                Depois")
  sm.qqplot(df_target[metric], ax=ax1)
  sm.qqplot(dfs_target[metric][metric], ax=ax2)
  plt.show()